>### LeNet-5

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers

#### Data

In [28]:
numbers = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = numbers.load_data()

In [29]:
train = tf.data.Dataset.from_tensor_slices((X_train, y_train)) 
test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [30]:
def preprocess(image, label): 
    image = tf.cast(image, tf.float32) 
    image = tf.expand_dims(image, -1) # (b,h,w,1) 
    image = tf.image.resize(image, [32, 32]) #resize前默认图片dim=3
    image = image/255.0 
    return image, label 

In [31]:
train = train.map(preprocess).shuffle(1024).batch(64).prefetch(-1)
test = test.map(preprocess).batch(64)
train
test

<PrefetchDataset shapes: ((None, 32, 32, 1), (None,)), types: (tf.float32, tf.uint8)>

<BatchDataset shapes: ((None, 32, 32, 1), (None,)), types: (tf.float32, tf.uint8)>

#### Model
Paper: [Gradient-Based Learning Applied to Document Recognition](http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf)

<img src='https://miro.medium.com/max/700/1*aQA7LuLJ2YfozSJa0pAO2Q.png'></img>

In [38]:
model = keras.Sequential([
        layers.Conv2D(6, 5, activation='sigmoid', input_shape=(32,32,1)),
        layers.AvgPool2D(2,2),
        layers.Conv2D(16, 5, activation='sigmoid'),
        layers.AvgPool2D(2,2),
        layers.Flatten(),
        layers.Dense(120, activation='sigmoid'),
        layers.Dense(84, activation='sigmoid'),
        layers.Dense(10, activation='sigmoid')
], name='LeNet-5')
model.summary()

Model: "LeNet-5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d_6 (Average (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_7 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_10 (Dense)             (None, 84)                1016

In [26]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train, validation_data=test, epochs=3)

Epoch 1/3
938/938 [==============================] - 31s 32ms/step - loss: 0.9166 - accuracy: 0.7087 - val_loss: 0.2903 - val_accuracy: 0.9144
Epoch 2/3
938/938 [==============================] - 32s 34ms/step - loss: 0.2362 - accuracy: 0.9287 - val_loss: 0.1853 - val_accuracy: 0.9423
Epoch 3/3
938/938 [==============================] - 32s 34ms/step - loss: 0.1580 - accuracy: 0.9517 - val_loss: 0.1220 - val_accuracy: 0.9624
